In [1]:
from bs4 import BeautifulSoup
import urllib.request
import csv

In [2]:
#specify url
urlpage =  'http://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'
#query the website

page=urllib.request.urlopen(urlpage)
soup=BeautifulSoup(page,'html.parser')

In [4]:
print(soup)

<!-- Template Name: League Table page
-->
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"> <![endif]-->
<!--[if gt IE 9]><!--><html class="no-js" lang="en-GB" prefix="og: http://ogp.me/ns#"><!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link data-minify="1" href="http://www.fasttrack.co.uk/wp-content/cache/min/1/ab1f96866a1c75dc57e29277e69b7831.css" rel="stylesheet"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet) & Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>
		League table - Fast Track	</ti

In [5]:
#find result
table=soup.find('table',attrs={'class':'tableSorter'})
results=table.find_all('tr')
print('Number of results',len(results))

Number of results 101


In [6]:
print(results)

[<tr>
<th>Rank</th>
<th>Company</th>
<th class="">Location</th>
<th class="no-word-wrap">Year end</th>
<th class="" style="text-align:right;">Annual sales rise over 3 years</th>
<th class="" style="text-align:right;">Latest sales £000s</th>
<th class="" style="text-align:right;">Staff</th>
<th class="">Comment</th>
<!--				<th>FYE</th>-->
</tr>, <tr>
<td>1</td>
<td><a href="http://www.fasttrack.co.uk/company_profile/plan-com/"><span class="company-name">Plan.com</span></a>Communications provider</td>
<td>Isle of Man</td>
<td>Sep-17</td>
<td style="text-align:right;">364.38%</td>
<td style="text-align:right;">*35,418</td>
<td style="text-align:right;">90</td>
<td>About 650 partners use its telecoms platform to support more than 100,000 UK business customers</td>
<!--						<td>Sep-17</td>-->
</tr>, <tr>
<td>2</td>
<td><a href="http://www.fasttrack.co.uk/company_profile/psioxus-2/"><span class="company-name">PsiOxus</span></a>Biotechnology developer</td>
<td>Oxfordshire</td>
<td>Dec-17</t

In [7]:
# create and write headers to a list 
rows = []
rows.append(['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments']]


In [9]:
# loop over results
for result in results:
    # find all columns per result
    data = result.find_all('td')
    # check that columns have data 
    if len(data) == 0: 
        continue
    # write columns to variables
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearend = data[3].getText()
    salesrise = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()
    
    # extract description from the name
    companyname = data[1].find('span', attrs={'class':'company-name'}).getText()    
    description = company.replace(companyname, '')
    
    # remove unwanted characters
    sales = sales.strip('*').strip('†').replace(',','')
     # go to link and extract company website
    url = data[1].find('a').get('href')
    page = urllib.request.urlopen(url)
    # parse the html 
    soup = BeautifulSoup(page, 'html.parser')
    # find the last result in the table and get the link
    try:
        tableRow = soup.find('table').find_all('tr')[-1]
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None
        
       # write each result to rows
    rows.append([rank, company, webpage, description, location, yearend, salesrise, sales, staff, comments])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'], ['1', 'Plan.comCommunications provider', 'http://www.plan.com', 'Communications provider', 'Isle of Man', 'Sep-17', '364.38%', '35418', '90', 'About 650 partners use its telecoms platform to support more than 100,000 UK business customers'], ['2', 'PsiOxusBiotechnology developer', 'http://www.psioxus.com', 'Biotechnology developer', 'Oxfordshire', 'Dec-17', '311.67%', '53136', '54', 'Received a $15m milestone payment from its development partner Bristol-Myers Squibb in December'], ['3', 'CensorNetCloud security software developer', 'http://www.censornet.com', 'Cloud security software developer', 'Basingstoke', 'Dec-17', '210.17%', '7535', '77', 'Has more than 4,000 customers, including McDonald’s in France and the European Central Bank'], ['4', 'thoughtonomyAutomation software developer', 'http://www.thoughtonomy.com', 'Automation software d

In [ ]:
# Create csv and write rows to output file
with open('techtrack100.csv','w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)